<a href="https://colab.research.google.com/github/sks01dev/Fast.ai-/blob/main/Tabular_Data_Analysis_using_FastAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai

In [6]:
# fetch the libraries
from fastai.tabular.all import *

In [5]:
# load the data
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/models'),Path('/root/.fastai/data/adult_sample/adult.csv'),Path('/root/.fastai/data/adult_sample/export.pkl')]

In [7]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [14]:
df.shape

(32561, 15)

In [8]:
df.select_dtypes(include="object").columns

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country', 'salary'],
      dtype='object')

In [9]:
# Create the data loader
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names='salary',
                                  cat_names=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                                  cont_names = ['age', 'fnlwgt', 'education-num'],
                                  procs=[Categorify, FillMissing, Normalize])

/usr/local/lib/python3.12/dist-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [10]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,False,49.0,40000.003019,13.0,<50k
1,Private,HS-grad,Never-married,Sales,Other-relative,Black,False,22.0,412315.988951,9.0,<50k
2,Federal-gov,Masters,Never-married,Tech-support,Not-in-family,White,False,40.0,406463.011109,14.0,<50k
3,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,57.0,161964.001400,9.0,<50k
4,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,False,50.0,139703.001070,10.0,>=50k
5,Private,11th,Divorced,Other-service,Unmarried,White,False,32.0,185731.999814,7.0,<50k
6,Local-gov,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,False,44.0,65144.993916,9.0,<50k
7,Private,Assoc-voc,Never-married,Adm-clerical,Not-in-family,White,False,25.0,254745.999045,11.0,<50k
8,Private,Doctorate,Married-civ-spouse,Prof-specialty,Husband,White,False,47.0,141511.000828,16.0,>=50k
9,Private,HS-grad,Divorced,Other-service,Not-in-family,White,False,27.0,109997.001600,9.0,<50k


In [11]:
# We can also do using TabularPandas but we need the split information

splits = RandomSplitter(valid_pct=0.2)(range_of(df))
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits)

/usr/local/lib/python3.12/dist-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [12]:
dls = to.dataloaders(bs=64)

In [13]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,HS-grad,Never-married,Sales,Not-in-family,White,False,25.000000,322585.003212,9.0,<50k
1,Private,Some-college,Married-civ-spouse,Transport-moving,Husband,White,False,40.000000,206048.999776,10.0,<50k
2,State-gov,Some-college,Divorced,Exec-managerial,Not-in-family,White,False,66.000001,71075.005647,10.0,<50k
3,Private,Some-college,Never-married,Craft-repair,Not-in-family,White,False,39.000000,32146.004718,10.0,<50k
4,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,29.000000,29973.998809,9.0,<50k
5,Private,Some-college,Never-married,Sales,Own-child,Black,False,25.000000,195913.999968,10.0,<50k
6,Private,Some-college,Divorced,Craft-repair,Unmarried,White,False,36.000000,229180.000767,10.0,<50k
7,?,Some-college,Never-married,?,Own-child,White,False,22.000000,121070.002849,10.0,<50k
8,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,39.000000,174242.000199,9.0,<50k
9,Private,Some-college,Divorced,Sales,Not-in-family,White,False,45.000000,240355.999211,10.0,<50k


In [15]:
learn = tabular_learner(dls, metrics=accuracy)

In [18]:
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.348248,0.365678,0.832770,00:04
1,0.337251,0.362571,0.832924,00:04
2,0.346168,0.359142,0.832924,00:05


In [19]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5.0,10.0,1.0,5.0,5.0,5.0,1.0,0.328775,2.926030,1.149855,0.0,0.0
1,5.0,10.0,3.0,5.0,1.0,5.0,1.0,-0.331470,0.187321,1.149855,1.0,1.0
2,1.0,16.0,7.0,1.0,2.0,5.0,1.0,2.236151,4.164874,-0.028313,0.0,0.0
3,1.0,7.0,5.0,1.0,4.0,5.0,1.0,-1.578601,0.819234,-1.991928,0.0,0.0
4,5.0,12.0,3.0,4.0,1.0,5.0,1.0,0.622218,-0.200202,-0.421036,0.0,0.0
5,5.0,10.0,3.0,13.0,1.0,5.0,1.0,0.548857,1.698559,1.149855,1.0,1.0
6,5.0,10.0,1.0,13.0,2.0,5.0,1.0,-0.258110,-0.021007,1.149855,0.0,0.0
7,5.0,9.0,3.0,4.0,1.0,5.0,1.0,0.989021,-0.363978,0.364410,0.0,1.0
8,5.0,16.0,3.0,2.0,6.0,5.0,1.0,0.182054,-0.199131,-0.028313,1.0,1.0


In [25]:
df[:5]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [29]:
row, clas, prob = learn.predict(df.iloc[0])
row.show()

/usr/local/lib/python3.12/dist-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101320.00177,12.0,>=50k


In [31]:
clas, prob

(tensor(1), tensor([0.4691, 0.5309]))

In [32]:
# get prediction to a new df
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

/usr/local/lib/python3.12/dist-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [40]:
preds, _ = learn.get_preds(dl=dl)
pred_labels = preds.argmax(dim=1)
pd.DataFrame({
    'id': test_df.index,
    'salary': pred_labels.numpy()}).to_csv('submission.csv', index=False)


In [41]:
df1 = pd.read_csv('submission.csv')
df1.head()

,id,salary
0,0,1
1,1,1
2,2,0
3,3,1
4,4,0
